In [56]:
import pandas as pd

# Create `goals_df`

In [57]:
def fix_name(name):
    if name in name_fixes:
        return name_fixes[name]
    else:
        return name

name_fixes = {
    "Corey Taylor": "Corey Blackett-Taylor",
    "Craig Carl Curran": "Craig Curran",
    "Chris Edwards": "Christian Edwards",
    "Dave Nugent": "David Nugent",
    "Dylan Mottley Henry": "Dylan Mottley-Henry",
    "Ian Thomas-Moore": "Ian Moore",
    "John-Louis Akpa Akpro": "Jean-Louis Akpa Akpro",
    "John Morrissey": "Johnny Morrissey",
    "Jonathon Margetts": "Johnny Margetts",
    "Joseph Maguire": "Joe Maguire",
    "Kaylden Brown": "Kayleden Brown",
    "Lewis Sinnot": "Lewis Sinnott",
    "Lateef Elford Alliyu": "Lateef Elford-Alliyu",
    "Michael Jackson": "Mike Jackson",
    "Richard Sutton": "Ritchie Sutton",
    "Robert Taylor": "Rob Taylor",
    "Samuel Taylor": "Sam Taylor",
    "Steven O'Leary": "Stephen O'Leary"
}

In [58]:
managers_df = pd.read_csv("https://raw.githubusercontent.com/petebrown/complete-record/main/output/managers.csv", parse_dates=["date_from", "date_to"])

managers_df.tail(3)

,manager_name,date_from,date_to,role
48,Micky Mellon,2021-06-01,2023-03-19,Manager
49,Ian Dawes,2023-03-20,2023-05-04,Caretaker
50,Ian Dawes,2023-05-04,2023-08-02,Manager


In [59]:
managers_df.to_csv('data/managers.csv', index = False)

In [60]:
def get_manager(game_date):
    try:
        return " & ".join(managers_df[(managers_df.date_from <= game_date) & (managers_df.date_to >= game_date)].manager_name)
    except:
        return None

results = pd.read_csv("https://raw.githubusercontent.com/petebrown/update-results/main/data/results_df.csv", parse_dates = ["game_date"])

results["manager"] = results.game_date.apply(get_manager)

results.loc[results.goals_for > results.goals_against, "outcome"] = "W"
results.loc[results.goals_for == results.goals_against, "outcome"] = "D"
results.loc[results.goals_for < results.goals_against, "outcome"] = "L"

results.head(1)

,season,game_date,opposition,venue,score,home_team,away_team,outcome,home_goals,away_goals,...,attendance,stadium,league_tier,generic_comp,game_type,goal_diff,ssn_game_no,ssn_comp_game_no,weekday,manager
0,2022/23,2023-05-08,Northampton Town,H,0-1,Tranmere Rovers,Northampton Town,L,0.0,1.0,...,8225.0,Prenton Park,4.0,Football League,League,-1.0,53,46,Monday,Ian Dawes


In [61]:
cup_game_details = pd.read_csv("https://raw.githubusercontent.com/petebrown/complete-record/main/11v11-extra-details/cup_details.csv", parse_dates = ["game_date"]).rename(columns = {"extra_time": "aet"})
cup_game_details.cup_replay = cup_game_details.cup_replay.replace(0, "")
cup_game_details.aet = cup_game_details.aet.replace(0, "")

cup_game_details.head(3)

,game_date,ko_time,cup_round,cup_leg,cup_stage,cup_replay,cup_section,aet,pen_outcome,pen_score,pen_gf,pen_ga,agg_outcome,agg_score,agg_gf,agg_ga,away_goal_outcome,gg_outcome
0,2022-11-22,7.00pm,2,NaN,2nd round - northern section,,Northern,,L,4-5,5.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-11-05,3.00pm,1,NaN,1st round,,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-10-18,7.00pm,G,NaN,Group B - North,,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [62]:
sb_match_apps = pd.read_csv(
    "https://raw.githubusercontent.com/petebrown/update-player-stats/main/data/players_df.csv",
    parse_dates = ["game_date"]
).rename(columns = {
    "sb_game_id": "game_id",
    "sb_player_id": "player_id",
})

sb_match_apps.game_id = sb_match_apps.game_id.str.replace("tpg", "").astype(int)

sb_match_apps.player_name = sb_match_apps.player_name.apply(fix_name)

In [63]:
sb_game_ids = sb_match_apps[["game_id", "game_date", "season"]].drop_duplicates().sort_values(by = ["game_date"]).reset_index(drop = True)

sb_game_ids.head(3)

,game_id,game_date,season
0,240309,1996-08-17,1996/97
1,243200,1996-08-21,1996/97
2,242383,1996-08-23,1996/97


In [64]:
sb_player_ids = sb_match_apps[["player_id", "player_name"]].drop_duplicates().reset_index(drop = True)
sb_player_ids.player_name = sb_player_ids.player_name.apply(fix_name)

sb_player_ids.head(3)

,player_id,player_name
0,128729,Jake Burton
1,18605,Peter Clarke
2,103555,Josh Dacres-Cogley


In [65]:
game_nos = results[["game_date", "season", "ssn_game_no"]].rename(columns = {"ssn_game_no": "game_no"}).sort_values(["game_date"]).reset_index(drop = True)

game_nos.head(3)

,game_date,season,game_no
0,1921-08-27,1921/22,1
1,1921-09-03,1921/22,2
2,1921-09-10,1921/22,3


### Import goal details scraped from Soccerbase

**Seasons:** 1996-97 - 2022/23

**Fields:**
- `player_name`
- `minute`
- `penalty` (0/1)
- `own_goal` (0/1)
- `game_date` _(after join)_

In [66]:
# Import Soccerbase goal details
sb_goals = pd.read_csv("https://raw.githubusercontent.com/petebrown/scrape-goals/main/data/goals.csv")

# Filter for Tranmere goals
sb_goals = sb_goals[sb_goals["goal_type"] == "for"]

# Fix player names
sb_goals.player_name = sb_goals.player_name.apply(fix_name)

# Add game dates via a join
sb_goals = sb_goals.merge(sb_game_ids, on = "game_id", how = "left").sort_values(["game_date", "minute"])

# Reduce to subset of columns
sb_goals = sb_goals[["game_date", "player_name", "minute", "penalty", "own_goal"]]

sb_goals.head(3)

,game_date,player_name,minute,penalty,own_goal
1713,1996-08-17,Johnny Morrissey,22,0,0
1714,1996-08-21,John Aldridge,30,0,0
1715,1996-08-21,Ivano Bonetti,56,0,0


### Import goals from Complete Record

**Seasons:** 1921/22 - 1995/96 (after filter)

**Fields:**

* `game_date`
* `player_name`
* `goals_scored` (no. of goals scored by player in game)

In [67]:
# Import Complete Record goal details and convert to a dictionary
all_goals = pd.read_csv("https://raw.githubusercontent.com/petebrown/complete-record/main/output/scorers-long.csv").merge(game_nos, how = "left", on = ["season", "game_no"])[["season", "game_date", "player_name", "goals_scored"]].to_dict("records")

# Initiate empty list for goals
cr_goals = []

# Loop through each goal and one for every goal in goals_scored column
for goal in all_goals:
    n_goals = goal["goals_scored"]

    for i in range(n_goals):
        cr_goals.append(goal)

# Convert to Pandas dataframe
cr_goals = pd.DataFrame(cr_goals).drop("goals_scored", axis = 1)

# Add own_goal column to match sb_goals
cr_goals.loc[cr_goals.player_name == "OG", "own_goal"] = 1
cr_goals.loc[cr_goals.player_name != "OG", "own_goal"] = 0
cr_goals.own_goal = cr_goals.own_goal.astype(int)

# Filter for seasons covered by Soccerbase data, i.e. before 1996/97
cr_goals = cr_goals[cr_goals.season < "1996"]

# Drop the season column
cr_goals = cr_goals.drop("season", axis = 1)

cr_goals.head(3)

,game_date,player_name,own_goal
0,1921-08-27,Tom Stuart,0
1,1921-08-27,Charles Milnes,0
2,1921-08-27,Fred Groves,0


### Import manually collected goal minutes

In [68]:
# Import manually collected goal minute data
cr_goal_mins = pd.read_csv("https://raw.githubusercontent.com/petebrown/complete-record/main/seasons/manual/goal_mins.csv", parse_dates = ["game_date"]).sort_values(["game_date", "goal_min"])

# Amend penalty column to 0/1
cr_goal_mins.penalty = cr_goal_mins.penalty.apply(lambda x: 1 if x == "pen" else 0)

# Add goal numbers for each player in each game, e.g. 1-3 for a hat-trick
cr_goal_mins["pl_goal_no"] = cr_goal_mins.groupby(["player_name", "game_date"]).cumcount() + 1

cr_goal_mins.head(3)

,game_date,player_name,goal_min,penalty,pl_goal_no
0,1964-09-04,John Manning,60,0,1
1,1986-09-27,Johnny Morrissey,50,0,1
2,1986-09-27,Gary Williams,54,0,1


### Join manually collected goal minutes to Complete Record goal data

In [69]:
# For join: add goal numbers for each player in each game, e.g. 1-3 for a hat-trick
cr_goals["pl_goal_no"] = cr_goals.groupby(["player_name", "game_date"]).cumcount() + 1

# Merge goal minutes with Complete Record goal data and drop temporary pl_goal_no column
cr_goals = cr_goals.merge(cr_goal_mins, how = "left", on = ["game_date", "player_name", "pl_goal_no"]).drop(columns = ["pl_goal_no"])

cr_goals.head(3)

,game_date,player_name,own_goal,goal_min,penalty
0,1921-08-27,Tom Stuart,0,NaN,NaN
1,1921-08-27,Charles Milnes,0,NaN,NaN
2,1921-08-27,Fred Groves,0,NaN,NaN


### Import FA Trophy goals

**Fields:**

* `game_date`
* `player_name`
* `minute`
* `penalty` (0/1)
* `own_goal` (0/1)

In [70]:
fa_trophy_goals = pd.read_csv("https://raw.githubusercontent.com/petebrown/complete-record/main/fa-trophy/fa_trophy_goals.csv", parse_dates = ["game_date"])

fa_trophy_goals.head(3)

,game_date,player_name,minute,penalty,own_goal
0,2015-12-12,James Norwood,45,0,0
1,2015-12-12,James Norwood,82,0,0
2,2016-12-10,Ritchie Sutton,26,0,0


In [71]:
# Concatenate Complete Record and Soccerbase goal dataframes
goals_df = pd.concat([cr_goals, sb_goals, fa_trophy_goals], axis = 0)

goals_df = goals_df.sort_values(by = ["game_date", "minute"])[["game_date", "player_name", "goal_min", "penalty", "own_goal"]]

goals_df.head(3)

,game_date,player_name,goal_min,penalty,own_goal
0,1921-08-27,Tom Stuart,NaN,NaN,0
1,1921-08-27,Charles Milnes,NaN,NaN,0
2,1921-08-27,Fred Groves,NaN,NaN,0


#### Save goals to CSV

In [72]:
goals_df.to_csv('data/goals.csv', index=False)

# Create `subs_df`

In [73]:
season_dates = results[["game_date", "season"]].drop_duplicates().sort_values("game_date").reset_index(drop=True)

season_dates.head(3)

,game_date,season
0,1921-08-27,1921/22
1,1921-09-03,1921/22
2,1921-09-10,1921/22


In [74]:
game_nos = season_dates.copy()
game_nos["game_no"] = game_nos.sort_values("game_date").groupby("season").cumcount() + 1

game_nos.head(3)

,game_date,season,game_no
0,1921-08-27,1921/22,1
1,1921-09-03,1921/22,2
2,1921-09-10,1921/22,3


In [75]:
sb_game_ids.head(3)

,game_id,game_date,season
0,240309,1996-08-17,1996/97
1,243200,1996-08-21,1996/97
2,242383,1996-08-23,1996/97


In [76]:
sb_player_ids.head(3)

,player_id,player_name
0,128729,Jake Burton
1,18605,Peter Clarke
2,103555,Josh Dacres-Cogley


In [77]:
# Import Soccerbase subs and red card CSV
sb_sub_mins = pd.read_csv("https://raw.githubusercontent.com/petebrown/scrape-events/main/data/subs-and-reds.csv").merge(sb_game_ids, how = "left", on = "game_id") \
    .merge(sb_player_ids, how = "left", on = "player_id") \
    .sort_values("game_date") \
    [["game_date", "player_name", "min_on", "min_off"]].reset_index(drop = True)

# Import manual fixes to Soccerbase sub data
sb_sub_min_fixes = pd.read_csv("https://raw.githubusercontent.com/petebrown/complete-record/main/seasons/manual/sb_sub_fixes.csv", parse_dates = ["game_date"])[["game_date", "player_name", "min_on", "min_off"]]

# Filter out sendings off
sb_sub_mins = sb_sub_mins[~((sb_sub_mins.min_on.isna()) & (sb_sub_mins.min_off.isna()))]

# Filter out records for dates with manual fixes
sb_sub_mins = sb_sub_mins[~sb_sub_mins.game_date.isin(sb_sub_min_fixes.game_date.unique())]

# Append manual fixes to Soccerbase sub data
sb_sub_mins = pd.concat([sb_sub_mins, sb_sub_min_fixes]).sort_values("game_date").reset_index(drop = True)

sb_sub_mins.head(3)

,game_date,player_name,min_on,min_off
0,1996-08-17,Kenny Irons,NaN,69.0
1,1996-08-17,John Aldridge,NaN,79.0
2,1996-08-17,Johnny Morrissey,NaN,86.0


In [78]:
cr_sub_mins = pd.read_csv("https://raw.githubusercontent.com/petebrown/complete-record/main/output/cr_subs_and_reds.csv", parse_dates = ["game_date"], na_values = ["0"])[["game_date", "player_name", "min_off", "min_on"]]

cr_sub_mins.head(3)

,game_date,player_name,min_off,min_on
0,1965-08-23,Jack Lornie,NaN,75.0
1,1965-08-23,Mandy Hill,75.0,NaN
2,1965-10-15,Eddie Stuart,82.0,NaN


In [79]:
sub_mins_df = pd.concat([cr_sub_mins, sb_sub_mins]).sort_values("game_date").reset_index(drop = True)

sub_mins_df.head(3)

,game_date,player_name,min_off,min_on
0,1965-08-23,Jack Lornie,NaN,75.0
1,1965-08-23,Mandy Hill,75.0,NaN
2,1965-10-15,Eddie Stuart,82.0,NaN


# Create `shirt_nos_df`

In [80]:
sub_mins_df.to_csv('data/sub_mins.csv', index = False)

In [81]:
shirt_nos_df = pd.read_csv("https://raw.githubusercontent.com/petebrown/complete-record/main/squad_nos/squad_nos.csv")

shirt_nos_df.head(3)

,season,squad_no,player_name
0,1999/00,1,John Achterberg
1,1999/00,2,Graham Allen
2,1999/00,3,Andy Thompson


# Create `red_cards_df`

In [82]:
cr_red_cards = pd.read_csv("https://raw.githubusercontent.com/petebrown/complete-record/main/seasons/manual/cards_red.csv", parse_dates = ["game_date"]).drop(columns = ["red_cards"])

cr_red_cards.head(3)

,game_date,player_name,min_so
0,1986-09-27,Andy Thorpe,90
1,1986-10-11,Frank Worthington,77
2,1987-02-14,Simon Farnworth,87


In [83]:
sb_red_cards = pd.read_csv("https://raw.githubusercontent.com/petebrown/scrape-events/main/data/subs-and-reds.csv").merge(sb_game_ids, how = "left", on = "game_id").merge(sb_player_ids, how = "left", on = "player_id")[["game_date", "player_name", "min_so"]].sort_values("game_date")

sb_red_cards = sb_red_cards[~sb_red_cards.min_so.isna()].reset_index(drop = True)

sb_red_cards.head(3)

,game_date,player_name,min_so
0,1996-08-17,Alan Rogers,69.0
1,1997-01-14,John Aldridge,78.0
2,1997-01-18,Dave Higgins,34.0


In [84]:
fa_trophy_reds = pd.read_csv("https://raw.githubusercontent.com/petebrown/complete-record/main/fa-trophy/fa_trophy_cards.csv", parse_dates = ["game_date"])[["game_date", "player_name", "red_card", "min_so"]].query("red_card == 1").drop(columns = "red_card").reset_index(drop = True)

fa_trophy_reds.head(3)

,game_date,player_name,min_so
0,2015-10-27,Scott Davies,14.0


In [85]:
red_cards_df = pd.concat([cr_red_cards, sb_red_cards, fa_trophy_reds], axis = 0).sort_values(["game_date", "min_so"]).reset_index(drop = True)

red_cards_df.head(3)

,game_date,player_name,min_so
0,1986-09-27,Andy Thorpe,90.0
1,1986-10-11,Frank Worthington,77.0
2,1987-02-14,Simon Farnworth,87.0


#### Save red_cards to CSV

In [86]:
red_cards_df.to_csv('data/red_cards.csv', index = False)

# Create `yellow_cards_df`

In [87]:
cr_yellows = pd.read_csv("https://raw.githubusercontent.com/petebrown/complete-record/main/seasons/manual/cards_yellow.csv", parse_dates = ["game_date"])

cr_yellows.head(3)

,game_date,player_name,yellow_cards
0,1986-09-27,Andy Thorpe,1
1,1986-09-27,Ronnie Moore,1
2,1986-09-27,Mark Hughes,1


In [88]:
sb_yellows = sb_match_apps[["game_date", "player_name", "yellow_cards"]].query("yellow_cards > 0").sort_values(by = "game_date")

sb_yellows.head(3)

,game_date,player_name,yellow_cards
17600,1996-08-17,Ivano Bonetti,1
17700,1996-08-21,Paul Cook,1
18130,1996-08-23,Shaun Teale,1


In [89]:
fa_trophy_yellows = pd.read_csv("https://raw.githubusercontent.com/petebrown/complete-record/main/fa-trophy/fa_trophy_cards.csv", parse_dates = ["game_date"])[["game_date", "player_name", "yellow_card"]].query("yellow_card == 1").rename(columns = {"yellow_card": "yellow_cards"})

fa_trophy_yellows.head(3)

,game_date,player_name,yellow_cards
0,2015-12-12,Lee Vaughan,1
1,2015-12-12,Lois Maynard,1
2,2016-12-10,Jeff Hughes,1


In [90]:
yellow_cards_df = pd.concat([cr_yellows, sb_yellows, fa_trophy_yellows], axis = 0).sort_values(by = "game_date").reset_index(drop = True)

yellow_cards_df.head(3)

,game_date,player_name,yellow_cards
0,1986-09-27,Andy Thorpe,1
1,1986-09-27,Ronnie Moore,1
2,1986-09-27,Mark Hughes,1


#### Save yellow_cards to CSV

In [91]:
yellow_cards_df.to_csv('data/yellow_cards.csv', index = False)

# Create `player_apps_df`

In [92]:
pl_ssns_9798 = pd.read_csv("https://raw.githubusercontent.com/petebrown/complete-record/main/output/player_ssns_9798.csv")
pl_ssns_9798.head(3)

,surname,forename,player_name,ssn,season,disam_name
0,Aldridge,John,John Aldridge,1997,1997/98,Aldridge
1,Branch,Graham,Graham Branch,1997,1997/98,Branch
2,Challinor,Dave,Dave Challinor,1997,1997/98,Challinor


In [93]:
pl_ssns_9899 = pd.read_csv("https://raw.githubusercontent.com/petebrown/complete-record/main/output/player_ssns_9899.csv")
pl_ssns_9899.head(3)

,surname,forename,player_name,ssn,season,disam_name
0,Simonsen,Steve,Steve Simonsen,1998,1998/99,Simonsen
1,Frail,Stephen,Stephen Frail,1998,1998/99,Frail
2,Thompson,Andy,Andy Thompson,1998,1998/99,Thompson


In [94]:
pl_ssns = pd.read_csv("https://raw.githubusercontent.com/petebrown/complete-record/main/output/player_ssns.csv")
pl_ssns.head(3)

,surname,forename,player_name,ssn,season,disam_name
0,A'Court,Alan,Alan A'Court,1964,1964/65,A'Court
1,A'Court,Alan,Alan A'Court,1965,1965/66,A'Court
2,Adams,Arthur,Arthur Adams,1932,1932/33,Adams A


In [95]:
cr_pl_ssns = pd.concat([pl_ssns, pl_ssns_9798, pl_ssns_9899], axis = 0).sort_values(by = ["surname", "forename", "ssn"]).reset_index(drop = True)
cr_pl_ssns.head(3)

,surname,forename,player_name,ssn,season,disam_name
0,A'Court,Alan,Alan A'Court,1964,1964/65,A'Court
1,A'Court,Alan,Alan A'Court,1965,1965/66,A'Court
2,Achterberg,John,John Achterberg,1998,1998/99,Achterberg


In [96]:
cr_pl_ssns_prepped = cr_pl_ssns[["season", "disam_name", "player_name"]].drop_duplicates().reset_index(drop = True)

cr_player_apps = pd.read_csv("https://raw.githubusercontent.com/petebrown/complete-record/main/output/apps_long.csv") \
    .merge(game_nos, how = "left", on = ["season", "game_no"]) \
    .merge(cr_pl_ssns_prepped, how = "left", left_on = ["season", "player_name"], right_on = ["season", "disam_name"]) \
    .rename(columns = {"player_name_y": "player_name"}) \
    [["game_date", "player_name", "shirt_no", "role"]]

cr_player_apps.head(3)

,game_date,player_name,shirt_no,role
0,1921-08-27,Harry Bradshaw,1,starter
1,1921-08-27,John Grainger,2,starter
2,1921-08-27,Tom Stuart,3,starter


In [97]:
fa_trophy_player_apps = pd.read_csv("https://raw.githubusercontent.com/petebrown/complete-record/main/fa-trophy/fa_trophy_fixtures.csv", parse_dates = ["game_date"]) \
    [["game_date", "player_name", "shirt_no", "role"]]

fa_trophy_player_apps.head(3)

,game_date,player_name,shirt_no,role
0,2015-12-12,Scott Davies,1,starter
1,2015-12-12,Lee Vaughan,7,starter
2,2015-12-12,Steve McNulty,24,starter


In [98]:
def get_squad_no(season, player_name, game_date):
    try:
        if season == "2014/15" and player_name == "Janoi Donacien" and game_date < pd.Timestamp("2015-03-07"):
            return 19
        elif season == "2014/15" and player_name == "Janoi Donacien" and game_date >= pd.Timestamp("2015-03-07"):
            return  12
        else:
            return shirt_nos_df[(shirt_nos_df.season == season) & (shirt_nos_df.player_name == player_name)].squad_no.values[0]
    except:
        return None

sb_player_apps = sb_match_apps[~sb_match_apps.season.isin(["1996/97", "1997/98", "1998/99"])][["game_date", "player_name"]].copy()

sb_player_apps["shirt_no"] = sb_match_apps.apply(lambda x: get_squad_no(x.season, x.player_name, x.game_date), axis = 1)

In [99]:
def get_role(game_date, player_name):
    subs = sub_mins_df[~sub_mins_df.min_on.isna()].drop(columns=['min_off', 'min_on'])

    subs = subs[(subs.game_date == game_date) & (subs.player_name == player_name)]

    if subs.empty:
        return "starter"
    else:
        return "sub"

sb_player_apps["role"] = sb_player_apps.apply(lambda x: get_role(x.game_date, x.player_name), axis=1)

sb_player_apps.head(3)

,game_date,player_name,shirt_no,role
0,2022-03-05,Jake Burton,27.0,starter
1,2021-08-07,Peter Clarke,26.0,starter
2,2021-08-10,Peter Clarke,26.0,starter


In [100]:
player_apps = pd.concat([cr_player_apps, sb_player_apps, fa_trophy_player_apps], axis = 0).sort_values(by = ["game_date", "role", "shirt_no"]).reset_index(drop = True)

player_apps.head(3)

,game_date,player_name,shirt_no,role
0,1921-08-27,Harry Bradshaw,1.0,starter
1,1921-08-27,John Grainger,2.0,starter
2,1921-08-27,Tom Stuart,3.0,starter


#### Save player_apps to csv

In [101]:
player_apps.to_csv('data/players_apps.csv', index = False)